#### Python Fundamentals for Data Analysts
## 4.8: Grouping Data & Aggregating Variables

### Content

1. Aggregation on average ORDER_NUMBER GroupBy DEPARTMENT_ID
2. Aggragation on maximum ORDER_NUMBER GroupBy USER_ID
3. Using Transform to assign the new aggregation to a new column
4. Create LOYALTY flag based on max order value
5. Create Spender Flag to identify Low Spender and High Spender
6. Create Order Frequency flag to identify user’s ordering behavior
    
Different ways to do Aggregation
- df_sub.groupby('department_id').agg({'order_number': ['mean']})
- df_sub.groupby('department_id')['order_number'].mean()
- df_combined['max_order'] = df_combined.groupby(['user_id'])['order_number'].transform(np.max)

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import os
from datetime import datetime

In [2]:
# Set path to access data files & Load data file
path = r'C:\Users\tsoew\OneDrive\Desktop\InstaCart Basket Analysis'

In [3]:
# Import previous version of ords_prods_combined data
df_combined = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ord_prod_combined_3.pkl'))

In [4]:
# Drop unused columns
df_combined.drop(axis=1, columns='_merge', inplace=True)

In [5]:
# Check the shape of the data
df_combined.shape

(32404859, 18)

In [6]:
# Create a subset of the dataframe
df_sub = df_combined[:1000000]

### Perform Aggregation using GROUPBY

#### Step 1: Aggregation on average ORDER_NUMBER GroupBy DEPARTMENT_ID

In [7]:
# Perform aggregation on SUBSET dataframe for AVERAGE order_number GROUPED by department_id
df_sub.groupby('department_id').agg({'order_number': ['mean']})

# In SQL would be: 
# SELECT AVG(order_number)
# GROUPBY department_id

,order_number
,mean
department_id,
1,15.577493
2,17.320781
3,16.084944
4,17.530458
5,14.763075
6,16.658449
7,17.031590
8,15.076662


In [8]:
# Perform a aggregation on ENTIRE dataframe for AVERAGE order_number GROUPED by department_id
df_combined.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


**Note** There is a slightly different result between the subset and the entire dataframe calculation on the aggregation of department_id.
But the difference is not that big and mostly fall within 10% of the first data.

In [9]:
# Different way to achieve the same result
# df_sub.groupby('department_id')['order_number'].mean()

#### Step 2: Aggragation on maximum ORDER_NUMBER GroupBy USER_ID

In [10]:
# Perform aggragation on SUBSET dataframe for MAXIMUM order_number GROUPED by user_id
df_sub.groupby('user_id').agg({'order_number': ['max']})

,order_number
,max
user_id,
1,10
2,14
3,10
4,3
7,3
...,...
206204,2
206206,36


#### Step 3: Using Transform  to assign the new aggregation to a new column

In [11]:
# Use transform function to assign the new result on max_order column
df_combined['max_order'] = df_combined.groupby(['user_id'])['order_number'].transform(np.max)


C:\Users\tsoew\AppData\Local\Temp\ipykernel_10948\1263093511.py:2: FutureWarning: The provided callable <function amax at 0x0000017AA96BDB20> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_combined['max_order'] = df_combined.groupby(['user_id'])['order_number'].transform(np.max)


#### Step 4: Create LOYALTY flag based on max order value

In [12]:
# Use the loc function to assign labels to the new column
df_combined.loc[df_combined['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_combined.loc[(df_combined['max_order'] <= 40) & (df_combined['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_combined.loc[df_combined['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [13]:
# Check data frequencymax_order
df_combined['loyalty_flag'].value_counts()

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [14]:
# Getting basic statistics for prices aggragated by loyalty_flag
df_combined.groupby('loyalty_flag').agg({'prices': ['max', 'min', 'mean']})

prices                
                      max  min       mean
loyalty_flag                             
Loyal customer    99999.0  1.0  10.386336
New customer      99999.0  1.0  13.294670
Regular customer  99999.0  1.0  12.495717

#### Step 5: Create SPENDER Flag to identify Low Spender and High Spender

Create a spending flag for each user based on the average price across all their orders using the following criteria:
If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [15]:
# Use transform function to assign the new result on average spending price for each user
df_combined['avg_spend'] = df_combined.groupby(['user_id'])['prices'].transform(np.mean)


C:\Users\tsoew\AppData\Local\Temp\ipykernel_10948\988947795.py:2: FutureWarning: The provided callable <function mean at 0x0000017AA96BE840> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_combined['avg_spend'] = df_combined.groupby(['user_id'])['prices'].transform(np.mean)


In [16]:
# Use the loc function to assign labels to the new column
df_combined.loc[df_combined['avg_spend'] >= 10, 'spend_flag'] = 'High spender'
df_combined.loc[df_combined['avg_spend'] < 10, 'spend_flag'] = 'Low spender'

In [17]:
df_combined['spend_flag'].value_counts()

spend_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

#### Step 6: Create FREQUENCY flag to identify user’s ordering behavior

In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

- If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
- If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
- If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [18]:
# Use transform function to get the median days_since_prior_order for each user
df_combined['median_order_day'] = df_combined.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


C:\Users\tsoew\AppData\Local\Temp\ipykernel_10948\281221813.py:2: FutureWarning: The provided callable <function median at 0x0000017AA97FB740> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df_combined['median_order_day'] = df_combined.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [19]:
# Apply the condition to identify customer frequency
df_combined.loc[df_combined['median_order_day'] > 20, 'freq_customer'] = 'Non-frequent customer'
df_combined.loc[(df_combined['median_order_day'] > 10) & (df_combined['median_order_day'] <= 20), 'freq_customer'] = 'Regular customer'
df_combined.loc[df_combined['median_order_day'] <= 10, 'freq_customer'] = 'Frequent customer'

In [20]:
df_combined ['freq_customer'].value_counts()

freq_customer
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
nan                             5
Name: count, dtype: int64

In [21]:
df_combined.shape

(32404859, 24)

In [22]:
# Now export the updated dataframe to a pkl file, and name it orders_products_combined_ver4.pkl

print ('Start time: ' + datetime.now().strftime("%H:%M:%S"))
df_combined.to_pickle(os.path.join(path, 'Data','Prepared Data', 'ord_prod_combined_4.pkl'))
print ('End time: ' + datetime.now().strftime("%H:%M:%S"))

Start time: 13:50:09
End time: 13:50:52


# The End